# Demo to test and show the functionality of the library

In [1]:
# Importing Functions
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TKAgg') # Needed to have figures display properly. 
import flirimageextractor
import Utils as u
import cv2

DEBUG=True

In [2]:
#filename = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092339.jpg"
filename = "C:\\Users\\Francisco\\Documents\\ThermalDataset\\Control\\Thermal-Images\\paciente_3\\flir\\control\\flir_20190907T092339.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044637_061.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044706_939.jpg"
flir = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir.process_image(filename, RGB=True)

In [3]:
therm = flir.get_thermal_np()
rgb_fullres = flir.get_rgb_np()
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(therm)
    plt.title('Raw Thermal Image')
    plt.subplot(1,2,2)
    plt.imshow(rgb_fullres)
    plt.title('RGB Full Resolution Image')
    plt.show(block='TRUE') 

In [4]:
colormap_image, Imsalida,temp_img_resized,image_copy = u.extract_images(flir,plot=0)

Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [5]:
#u.save_thermal_csv(temp_img_resized,"temperature_examples.csv",delimiter=";")

# second part of the demo get the perspective transformation between two images

In [6]:
#filename2 = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092409.jpg"
filename2 = "C:\\Users\\Francisco\\Documents\\ThermalDataset\\Control\\Thermal-Images\\paciente_3\\flir\\control\\flir_20190907T093339.jpg"
flir2 = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir2.process_image(filename2, RGB=True)
therm2 = flir2.get_thermal_np()
rgb_fullres2 = flir2.get_rgb_np()
colormap_image2, Imsalida2,temp_img_resized2,image_copy2 = u.extract_images(flir2,plot=0)
#u.save_thermal_csv(temp_img_resized2,"temperature_examples2.csv",delimiter=";")
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(temp_img_resized2)
    plt.title('temp_img_resized2')
    plt.subplot(1,2,2)
    plt.imshow(image_copy2)
    plt.title('image_copy2')
    plt.show(block='TRUE') 


Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [7]:


"""
def fethearing_bin_to_color(binary_image, color_image):
    # Operación de cierre morfológico con un kernel elíptico de tamaño 5x5
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    
    # Fethearing usando filtro guiado
    filtered_image = cv2.ximgproc.guidedFilter(color_image, closed_image, 20, eps=0.1)#5, eps=1e-4
    
    # Umbralización de la imagen filtrada
    _, thresholded_image = cv2.threshold(filtered_image,20, 255, cv2.THRESH_BINARY)
    
    return filtered_image
"""


'\ndef fethearing_bin_to_color(binary_image, color_image):\n    # Operación de cierre morfológico con un kernel elíptico de tamaño 5x5\n    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))\n    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)\n    \n    # Fethearing usando filtro guiado\n    filtered_image = cv2.ximgproc.guidedFilter(color_image, closed_image, 20, eps=0.1)#5, eps=1e-4\n    \n    # Umbralización de la imagen filtrada\n    _, thresholded_image = cv2.threshold(filtered_image,20, 255, cv2.THRESH_BINARY)\n    \n    return filtered_image\n'

In [8]:
import registration_simpleitk as sr

In [9]:

segmented_Feet,segmented_temps=u.Find_feets(image_copy,temp_img_resized,percentage=20)
if DEBUG:
    u.plot_images_and_thermal(segmented_Feet,segmented_temps)
segmented_Feet2,segmented_temps2=u.Find_feets(image_copy2,temp_img_resized2,percentage=20)
print(segmented_Feet2[1].shape)
if DEBUG:
    u.plot_images_and_thermal(segmented_Feet2,segmented_temps2)

(442, 206, 3)


In [10]:
#u.visualize_descriptors(segmented_Feet[0],segmented_Feet2[0])
#u.visualize_descriptors(segmented_Feet[1],segmented_Feet2[1])

"""
output_image,homography_matrix=u.matchbydescriptors(segmented_Feet[0],segmented_Feet2[0])
plt.figure(figsize=(10,5))
plt.imshow(output_image)
plt.title('registration between images')
plt.show(block='TRUE') 

output_image,homography_matrix=u.matchbydescriptors(segmented_Feet[1],segmented_Feet2[1])
plt.figure(figsize=(10,5))
plt.imshow(output_image)
plt.title('registration between images')
plt.show(block='TRUE') 
"""

"\noutput_image,homography_matrix=u.matchbydescriptors(segmented_Feet[0],segmented_Feet2[0])\nplt.figure(figsize=(10,5))\nplt.imshow(output_image)\nplt.title('registration between images')\nplt.show(block='TRUE') \n\noutput_image,homography_matrix=u.matchbydescriptors(segmented_Feet[1],segmented_Feet2[1])\nplt.figure(figsize=(10,5))\nplt.imshow(output_image)\nplt.title('registration between images')\nplt.show(block='TRUE') \n"

In [11]:
# using a different approach
"""
registered_image_a_array, registered_image_b_array, composition_image =sr.register_images(segmented_Feet[0],segmented_Feet2[0],'displaceMeth4.hdf5')
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(registered_image_a_array)
    plt.title('registered_image_a_array')
    plt.subplot(1,2,2)
    plt.imshow(registered_image_b_array)
    plt.title('registered_image_b_array')
    plt.show(block='TRUE') 
    
plt.figure(figsize=(10,5))
plt.imshow(composition_image)
plt.title('composition_image')
plt.show(block='TRUE') 
"""

"\nregistered_image_a_array, registered_image_b_array, composition_image =sr.register_images(segmented_Feet[0],segmented_Feet2[0],'displaceMeth4.hdf5')\nif DEBUG:\n    plt.figure(figsize=(10,5))\n    plt.subplot(1,2,1)\n    plt.imshow(registered_image_a_array)\n    plt.title('registered_image_a_array')\n    plt.subplot(1,2,2)\n    plt.imshow(registered_image_b_array)\n    plt.title('registered_image_b_array')\n    plt.show(block='TRUE') \n    \nplt.figure(figsize=(10,5))\nplt.imshow(composition_image)\nplt.title('composition_image')\nplt.show(block='TRUE') \n"

In [12]:
# using a different approach
output =sr.register_images_Similarity2DTransform(segmented_Feet[0],segmented_Feet2[0],'displaceMeth4_advanced.hdf5')
fixed_imagen = sr.sitk_image_to_opencv(output["fixed"])
moving_imagen = sr.sitk_image_to_opencv(output["moving"])
composition_image = sr.sitk_image_to_opencv(output["composition"])
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(fixed_imagen)
    plt.title('fixed_imagen')
    plt.subplot(1,2,2)
    plt.imshow(moving_imagen)
    plt.title('moving_imagen')
    plt.show(block='TRUE') 

plt.figure(figsize=(10,5))
plt.imshow(composition_image)
plt.title('composition_image')
plt.show(block='TRUE') 

thermalout= sr.cv2_grid_sampling(segmented_temps[0], segmented_temps2[0], output["Transform"], is_binary=False)

plt.figure(figsize=(10,5))
plt.imshow(thermalout)
plt.title('thermalout')
plt.show(block='TRUE') 



  0 =   -1.03654 : (0.9999753169421706, -5.0744374340444914e-05, 0.5744729815772023, 1.0667365853992907)
  1 =   -1.03920 : (0.9999702164752848, -0.00011948264503903722, 0.6472638099256308, 1.135304001548545)
  2 =   -1.04162 : (0.9999802234049079, -0.00020615293617565103, 0.7166499862868915, 1.207314771007606)
  3 =   -1.04385 : (0.9999970332315337, -0.0003068495952414722, 0.7821346863485935, 1.282890849467933)
  4 =   -1.04618 : (1.0000043566043704, -0.00041131895813448373, 0.8438159982836622, 1.3616017431897627)
  5 =   -1.04856 : (0.9999938858328509, -0.0005150499469524735, 0.9017283476128957, 1.4431256594684458)
  6 =   -1.05095 : (0.9999629081872006, -0.0006142620764821063, 0.9571268627397826, 1.5263782493325149)
  7 =   -1.05335 : (0.9999154409034293, -0.0007063998781490092, 1.0118813873828436, 1.6100557928468588)
  8 =   -1.05564 : (0.9998582551182142, -0.0007938445923980868, 1.06611974817678, 1.694068823393285)
  9 =   -1.05771 : (0.9997877649930514, -0.0008771233030000513, 1.

In [13]:

print(type(fixed_imagen))

print(fixed_imagen.shape)

print(fixed_imagen)

<class 'numpy.ndarray'>
(445, 206)
[[45 44 44 ... 39 40 40]
 [45 45 45 ... 39 40 40]
 [43 43 43 ... 40 40 40]
 ...
 [18 17 18 ... 13 13 13]
 [17 16 17 ... 13 13 13]
 [16 16 16 ... 13 13 13]]
